In [13]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv.main import load_dotenv

# .env 파일에서 환경변수 가져오기
load_dotenv()

# 공지사항 라우터 주소
notice_router = "https://www.anyang.ac.kr/main/communication/notice.do"

# 학사 공지사항 URL
school_url = 'https://www.anyang.ac.kr/main/communication/notice.do?mode=list&srSearchKey=&srSearchVal=&srCategoryId=18'

# AI융합학과 강의 정보가 담김 csv 경로
ai_csv_path = "../AI융합학과_utf.csv"
busi_csv_path = "../관광경영학과.csv"
com_csv_path = "../컴퓨터공학과.csv"


# 도구 정의
@tool
def get_notices() -> str:
    """안양대학교 학사 공지사항 목록을 반환한다."""
    resp = requests.get(school_url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    notice_divs = soup.find_all('div', {'class': "b-title-box"})

    notice_list = []
    for tr in notice_divs:
        link_tag = tr.find_all('a')[0]
        notice_list.append(f"\t{{\n\t\t제목: {link_tag.get('title')}, \n\t\t주소: {notice_router}{link_tag.get('href')}\n\t}}")

    return "[\n{0}\n]".format(",\n".join(notice_list))


@tool
def get_one_notice(notice_url: str) -> str:
    """특정 공지사항의 본문을 반환한다."""
    resp = requests.get(notice_url)
    soup = BeautifulSoup(resp.content, 'html.parser')
    notice_content = soup.find('div', {'class': 'b-main-box'})
    return notice_content.text.replace('\n\n', '').replace('\t', '')


@tool
def get_ai_table() -> str:
    """만양대학교 AI융합학과 강의 정보를 테이블 형태로 반환한다."""
    df = pd.read_csv(ai_csv_path)
    return df.to_string()


@tool
def get_busi_table() -> str:
    """만양대학교 관광경영학과 강의 정보를 테이블 형태로 반환한다."""
    df = pd.read_csv(busi_csv_path)
    return df.to_string()



@tool
def get_com_table() -> str:
    """만양대학교 컴퓨터공학과 강의 정보를 테이블 형태로 반환한다."""
    df = pd.read_csv(com_csv_path)
    return df.to_string()


# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# prompt 생성 (history 추가해야 함)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful anyang university assistant",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


# 이전에 정의한 도구 사용
tools = [get_notices, get_one_notice, get_ai_table, get_busi_table, get_com_table]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    # 추정 과정 출력
    verbose=True,
    handle_parsing_errors=True,
)

# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

# 질문
query = "동계 계절수업 공지에 대해 알려줘"

# 실행
# result = agent_with_chat_history.invoke({"input": query})
response = agent_with_chat_history.invoke(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

step_list =list()
# 출력 확인
# for step in response:
#     step_list.append(step)
#     print(step)

# 결과 출력
# print("-----------------------------------------------------------"
#       "실제 출력문:")
# print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `get_notices` with `{}`


[
	{
		제목: 범계역 청년출구 사업 관련 별정직 채용 공고 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34590&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: TEPS 시행으로 인한 수리관 일시사용중지 협조 요청[12/1] 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34582&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 2학기 학부 분납 4차 등록금 납부 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34541&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 2학기 대학원 분납 4차 등록금 납부 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34540&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 안양대학교 동계 계절수업 안내  자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34534&article.offset=0&articleLimit=15&srCate

In [14]:
query = "내가 아까 뭐라고 했지?"

# 실행
# result = agent_with_chat_history.invoke({"input": query})
response = agent_with_chat_history.invoke(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)



> Entering new AgentExecutor chain...
당신은 "동계 계절수업 공지에 대해 알려줘"라고 말씀하셨습니다.

> Finished chain.


In [15]:
query = "내가 아까 뭐라고 했지?"

# 실행
# result = agent_with_chat_history.invoke({"input": query})
response = agent_with_chat_history.invoke(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc1"}},
)



> Entering new AgentExecutor chain...
죄송하지만, 이전 대화 내용을 기억할 수 없습니다. 어떤 내용을 말씀하셨는지 다시 말씀해 주시면 도와드리겠습니다!

> Finished chain.


In [29]:
query = "정의현 교수님이 하시는 강의 목록 좀"

# 실행
# result = agent_with_chat_history.invoke({"input": query})
response = agent_with_chat_history.invoke(
    {"input": query},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)



> Entering new AgentExecutor chain...

Invoking: `get_com_table` with `{}`


    Unnamed: 0  개설학년    과목코드             과목명  분반 이수구분  교원명  학점  이론  실습                                  시간표    비고
0            0     1  NA2015          인공지능개론   1   전선  송인식   3   2   1                  금(1,2,3) 아리관(N)-301  타과제한
1            1     1  NA2015          인공지능개론   2   전선  송인식   3   2   1                  금(1,2,3) 아리관(N)-301  타과제한
2            2     1  NA2016         프로그래밍응용   1   전선  이상홍   3   1   2   화(6,7) 아리관(N)-607, 수(2) 아리관(N)-607  타과제한
3            3     1  NA2016         프로그래밍응용   2   전선  이상홍   3   1   2   화(8,9) 아리관(N)-607, 수(3) 아리관(N)-607  타과제한
4            4     2  NA3025       객체지향프로그래밍   1   전선  이상홍   3   1   2   월(4,5) 아리관(N)-301, 금(2) 아리관(N)-607  타과제한
5            5     2  NA3025       객체지향프로그래밍   2   전선  이상홍   3   1   2   월(6,7) 아리관(N)-301, 금(3) 아리관(N)-607  타과제한
6            6     2  NA4028        시스템프로그래밍   1   전선  권희용   3   1   2   수(3) 아리관(N)-301, 토(1,2) 아리관(N)-301  타과제한
7        

In [ ]:
"""
input : 동계 계절수업 공지에 대해 자세히 알려줘

[1m> Entering new AgentExecutor chain...[0m
[32;1m[1;3m
Invoking: `get_notices` with `{}`


[0m[36;1m[1;3m[
	{
		제목: 범계역 청년출구 사업 관련 별정직 채용 공고 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34590&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: TEPS 시행으로 인한 수리관 일시사용중지 협조 요청[12/1] 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34582&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 2학기 학부 분납 4차 등록금 납부 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34541&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 2학기 대학원 분납 4차 등록금 납부 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34540&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 안양대학교 동계 계절수업 안내  자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34534&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [장학]24년도 2학기 고졸 후학습자 장학사업(희망사다리Ⅱ유형) 신규장학생 2차 모집 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34530&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2025년 상반기 신임교원 초빙 공고 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34507&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [현장실습지원센터] 2024학년도 2학기 동계방학 현장실습생 모집 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34506&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: ISIC 국제학생증 발급비 지원행사 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34504&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [국제교류원] 2024학년도 동계방학 일본 단기 어학연수 참가자 모집 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34502&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [장학] 2025년 1학기 국가장학금 1차 신청 안내(필독) 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34500&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024 안양대 온라인 채용설명회 14 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34498&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [대학인권센터] 2024년 인권 교육 실시(학부 및 대학원생 대상)  자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34456&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [대학인권센터] 2024년 2차 폭력예방교육 실시(학부 및 대학원생 대상) 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34455&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [채용공고] 대아티아이 채용연계형 인턴 모집(~12/8) 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34441&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 안양대학교 공유기숙사 입주자 상시 모집 공고 및 호실별 내부 구조 사진 안내 (남자기숙사 입주대기자 모집) 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34428&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 국제 학생 학술연구 컨퍼런스(GSR) 행사 및 참가자 모집 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34424&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: [채용공고] (주)비디텍 인턴(채용연계형) 채용 공고(~11/29) 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34265&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 학칙 및 학칙시행세칙 개정안 공고 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34573&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 한세대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34509&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 명지대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34497&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 수원대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34496&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 평택대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34450&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 대진대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34432&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 협성대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34420&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 가톨릭대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34397&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 단국대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34395&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 한신대학교 학점교류 안내(전산회계실무_NCS 및 AWS클라우드서비스) 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34393&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 한경국립대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34374&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 수료생 2025년 2월 졸업신청 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34373&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 인천대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34372&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 경희대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34369&article.offset=0&articleLimit=15&srCategoryId=18
	},
	{
		제목: 2024학년도 동계계절학기 가천대학교 학점교류 안내 자세히 보기, 
		주소: https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34368&article.offset=0&articleLimit=15&srCategoryId=18
	}
][0m[32;1m[1;3m
Invoking: `get_one_notice` with `{'notice_url': 'https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34534&article.offset=0&articleLimit=15&srCategoryId=18'}`


[0m[33;1m[1;3m
[공지]
2024학년도 안양대학교 동계 계절수업 안내 
조회수
1498
작성자
학사지원과
작성일
2024.11.21
[서식15호]계절수업수강취소및환불신청서_240517.hwp
뷰어2024학년도 안양대학교 동계 계절수업 안내1. 수업기간 및 강의시간표가. 기간 : 2024. 12. 23(월) ~ 2025. 1. 15(수) 15일간(12/31(화)제외)나. 본 학기 계절수업은 대면수업으로 운영됩니다.다. 강의시간표 개설학과이수구분역량과목명학점담당교수수강학년수강인원강의시간 및강의실비고아리교양대학교양필수글로벌ESP:Critical Thinking3최선화전학년35수리관 50110:00~12:30영어회화Ⅰ대체교양선택문제해결엑셀을활용한사무자동화3김진수전학년35수봉관 90713:00~15:30문제해결청년심리학3윤덕규전학년35수리관 50510:00~12:30문제해결서사와매체3김중철전학년35수리관 50513:00~15:30글로벌드라마톡톡영어3손승희전학년35수리관 50213:00~15:30인성영성사회봉사2장유진전학년35수강료없음글로벌경영학과전공필수무역학개론3김태환2학년35수리관 40110:00~12:30마케팅원론3전주언2학년35수리관 30610:00~12:30전공선택무역영어3김태환3학년35수리관 40113:00~15:30중급회계Ⅰ3김승준2학년35수리관 30513:00~15:30원가관리회계3김승준3학년35수리관 30510:00~12:30현장실습Ⅰ3전주언3학년수강료없음현장실습Ⅱ6전주언3학년디지털미디어디자인학과전공선택현장실습Ⅰ3김진곤3,4학년수강료없음현장실습Ⅱ6김진곤3,4학년식품영양학과전공선택영양사현장실습2길복임3학년수강료없음화장품발명디자인학과전공선택라이브커머스마케팅캡스톤디자인3진화림3학년비812 10:00~12:30※수업계획서는 [포탈시스템(portal.anyang.ac.kr)-스마트종합정보-학생서비스-수업성적-강좌개설조회]에서 12월 3일(화)부터 확인 가능합니다.※현장실습 과목 수강을 원하는 학생들은 해당 학과에 신청하고, 학과는 현장실습센터를 통해 MOU체결 등 관련 절차를 거쳐 현장실습을 진행하시기 바랍니다.※2024학년도 2학기 현재 수강신청 중인 과목은 신청이 불가합니다. 2. 수강대상가. 재학생(휴학생은 수강 불가)3. 수강학점 범위가. 과목 수에 제한 없이 6학점까지 가능(졸업 시까지 21학점 수강 가능) 나. 사회봉사는 추가 신청이 가능 -계절수업 교과를 수강하지 않을 시 수강불가-계절수업 교과를 환불하여 수강하는 계절수업이 없을 경우 사회봉사 자동 취소4. 수강신청 기간 안내 가. 수강신청 기간 : 2024. 12. 9(월) 10:00 ~ 12. 10(화) 17:00(수강신청 사이트 : https://sugang.anyang.ac.kr)나. 폐강과목 공지 : 2024. 12. 11(수) 10:00 홈페이지 공지사항다. 수강신청 정정기간 : 2024. 12. 11(수) 13:00 ~ 12. 11(수) 17:00까지5. 성적처리가. 성적확인 및 이의신청 기간 : 2025. 1. 16(목) ~ 1.17(금) 15:00나. 성적확정 : 2025. 1. 20(월) 13:00 (예정)다. 3학점 과목은 상대평가Ⅰ, P과목은 Pass/Non-Pass 과목라. 계절수업 취득성적은 별도 표기되며, 1학기 석차, 장학생 선발 등 정규학기 성적에 영향 없음6. 수강료 금액 납부 및 등록가. 수강료 : 학점당 80,000(ex: 3학점 240,000원) 나. 등록기간 : 2024. 12. 13(금) 09:00 ~ 2024. 12. 19(목) 17:00(주말수납가능) ※ 추후 재무과 공지사항 참고하여 가상계좌 납부(등록문의 : 재무과 031-467-0747)다. 수강료를 납부하지 않을 경우 자동으로 수강 신청이 취소됩니다.7. 수강료 환불 안내가. 제출서류 : 계절수업 수강취소 및 환불신청서(별첨) 나. 수강료 환불금액 안내구분기준일자환불액수업개시 이전2024.12. 20. 17:00 까지수강료 전액총 수업일의 1/3 경과 전 2024.12. 30. 15:30 까지수강료의 2/3총 수업일의 1/2 경과 전2025.01. 03. 15:30 까지수강료의 1/2총 수업일의 1/2 경과 후2025.01. 03. 15:30 이후반환하지 않음다. 총 수업일의 1/2 경과 후 환불액이 없으며, 수강취소가 되지 않고 성적 부여됩니다.라. 제출방법 : 학사지원과(비504) 방문 제출 or 스캔하여 메일 제출(haksa@anyang.ac.kr) or 팩스 제출(031-467-0980)8. 수강신청 유의사항가. 최종 등록인원이 15명 미만인 강좌는 폐강될 수 있으며, 폐강강좌 공지 후 해당과목을 취소하지 않을 경우 자동 취소됩니다. - 수업개시 후 2일 안에 수강인원이 15명 미만인 경우 폐강될 수 있습니다.나. 결석 횟수가 3학점 기준 12차시 이상인 경우 학점 취득이 불가합니다.9. 재수강시 유의사항가. [ESP:Critical Thinking] 수강신청 시 재수강 혹은 대체수강이라고 표시되지 않을 경우 대체과목신청서를 작성 후 아리교양대학장(비전센터 707호)의 승인을 받아 학사지원과(비전센터 504호)에 제출하시기 바랍니다.나. C+ 이하인 과목에 한하여 재수강 가능다. 재수강 성적은 A이하로 제한라. 재수강 완료시 기존성적은 성적증명서에 R(Renounce)로 표기 10. 사회봉사, 현장실습 학점 : 계절수업 제한학점 6학점에 미포함=>사회봉사 관련일정은 사회봉사센터 공지사항 참조[안양대학교 홈페이지 상단 Search(돋보기 모양)-사회봉사 검색]가. 사회봉사 이수하지 못했을 경우 성적, 성적증명서에 NP로 표기나. 홈페이지에서 사회봉사 공지사항 검색 후, 첨부된 오리엔테이션 자료 및 제출양식을 내려받아 참고 바랍니다. 다. 온라인 소양교육 영상 시청 및 봉사활동 실시 후 제출양식을 정해진 기한 내 제출하셔야 사회봉사 학점이 인정됩니다.※ 사회봉사 문의 : 사회봉사센터 031-467-0847※ 현장실습 문의 : 현장실습지원센터 031-463-1278[0m[32;1m[1;3m2024학년도 안양대학교 동계 계절수업에 대한 안내는 다음과 같습니다:

### 1. 수업 기간 및 강의 시간표
- **기간**: 2024년 12월 23일(월) ~ 2025년 1월 15일(수) (12월 31일(화) 제외)
- **운영 방식**: 대면 수업으로 진행됩니다.

### 2. 수강 대상
- **재학생**만 수강 가능 (휴학생은 수강 불가)

### 3. 수강 학점 범위
- 과목 수에 제한 없이 최대 6학점까지 수강 가능 (졸업 시까지 21학점 수강 가능)
- 사회봉사는 추가 신청 가능

### 4. 수강 신청 기간
- **신청 기간**: 2024년 12월 9일(월) 10:00 ~ 12월 10일(화) 17:00
- **정정 기간**: 2024년 12월 11일(수) 13:00 ~ 17:00

### 5. 성적 처리
- 성적 확인 및 이의 신청 기간: 2025년 1월 16일(목) ~ 1월 17일(금) 15:00
- 성적 확정: 2025년 1월 20일(월) 13:00 (예정)

### 6. 수강료
- **금액**: 학점당 80,000원 (예: 3학점 240,000원)
- **등록 기간**: 2024년 12월 13일(금) 09:00 ~ 12월 19일(목) 17:00

### 7. 수강료 환불 안내
- 수업 시작 이전: 수강료 전액 환불
- 총 수업일의 1/3 경과 전: 수강료의 2/3 환불
- 총 수업일의 1/2 경과 전: 수강료의 1/2 환불
- 총 수업일의 1/2 경과 후: 환불 없음

### 8. 수강 신청 유의사항
- 최종 등록 인원이 15명 미만인 강좌는 폐강될 수 있습니다.
- 결석 횟수가 3학점 기준 12차시 이상인 경우 학점 취득 불가

### 9. 사회봉사 및 현장실습 학점
- 계절수업 제한 학점 6학점에 포함되지 않음

자세한 내용은 [여기](https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34534&article.offset=0&articleLimit=15&srCategoryId=18)에서 확인하실 수 있습니다.[0m

[1m> Finished chain.[0m
-----------------------------------------------------------실제 출력문:
2024학년도 안양대학교 동계 계절수업에 대한 안내는 다음과 같습니다:

### 1. 수업 기간 및 강의 시간표
- **기간**: 2024년 12월 23일(월) ~ 2025년 1월 15일(수) (12월 31일(화) 제외)
- **운영 방식**: 대면 수업으로 진행됩니다.

### 2. 수강 대상
- **재학생**만 수강 가능 (휴학생은 수강 불가)

### 3. 수강 학점 범위
- 과목 수에 제한 없이 최대 6학점까지 수강 가능 (졸업 시까지 21학점 수강 가능)
- 사회봉사는 추가 신청 가능

### 4. 수강 신청 기간
- **신청 기간**: 2024년 12월 9일(월) 10:00 ~ 12월 10일(화) 17:00
- **정정 기간**: 2024년 12월 11일(수) 13:00 ~ 17:00

### 5. 성적 처리
- 성적 확인 및 이의 신청 기간: 2025년 1월 16일(목) ~ 1월 17일(금) 15:00
- 성적 확정: 2025년 1월 20일(월) 13:00 (예정)

### 6. 수강료
- **금액**: 학점당 80,000원 (예: 3학점 240,000원)
- **등록 기간**: 2024년 12월 13일(금) 09:00 ~ 12월 19일(목) 17:00

### 7. 수강료 환불 안내
- 수업 시작 이전: 수강료 전액 환불
- 총 수업일의 1/3 경과 전: 수강료의 2/3 환불
- 총 수업일의 1/2 경과 전: 수강료의 1/2 환불
- 총 수업일의 1/2 경과 후: 환불 없음

### 8. 수강 신청 유의사항
- 최종 등록 인원이 15명 미만인 강좌는 폐강될 수 있습니다.
- 결석 횟수가 3학점 기준 12차시 이상인 경우 학점 취득 불가

### 9. 사회봉사 및 현장실습 학점
- 계절수업 제한 학점 6학점에 포함되지 않음

자세한 내용은 [여기](https://www.anyang.ac.kr/main/communication/notice.do?mode=view&articleNo=34534&article.offset=0&articleLimit=15&srCategoryId=18)에서 확인하실 수 있습니다.
"""